In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from pathlib import Path
from scipy.stats import ttest_rel

In [2]:
results_file = Path(r"../DVlog/trained_models/test_metrics.csv")

# load in the file with the results on the test set
df_results = pd.read_csv(results_file, sep=";")
df_results.head()

,Unnamed: 0,name,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
0,0,unimodal_bias_mf-group_mpnet_sent,0,0.929159,0.927273,0.926964,0.877117,0.953358,0.920030,0.900000,1.408046,0.900000,0.172414,1.000000,0.122449
1,1,unimodal_bias_mf-group_mpnet_sent,1,0.927386,0.927273,0.927192,0.877193,0.953522,0.919951,0.881111,1.351724,0.866667,0.137931,0.983607,0.102041
2,2,unimodal_bias_mf-group_mpnet_sent,1123,0.918511,0.915152,0.914619,0.859389,0.943915,0.910452,0.900000,1.448276,0.900000,0.206897,1.000000,0.142857
3,3,unimodal_bias_mf-group_mpnet_sent,3407,0.933333,0.933333,0.933299,0.894672,0.953522,0.938282,0.881111,1.013793,0.866667,0.103448,0.983607,0.102041
4,4,unimodal_bias_mf-group_mpnet_sent,42,0.934632,0.933333,0.933110,0.876737,0.962873,0.910543,0.948889,2.534483,0.933333,0.206897,0.983607,0.081633


In [13]:
# Performance metrics of the two models on the test set (e.g., F1-scores for each test sample)
base_model = "unimodal_mpnet_sent_keyw"
comparison_models = ["bimodal_mpnet_aud_sync_concat", "trimodal_mpnet_av_sync_keyw_concat"]
columns = ["f1", "Eq accuracy", "eq opportunity", "pred equality"]

# go over each comparison model and compute the t-test
base_info = df_results[df_results["name"] == base_model]
base_info

for comp_name in comparison_models:
    print(comp_name)
    comp_info = df_results[df_results["name"] == comp_name]
    
    # for each column perform the paired t-test
    for column in columns:
        model1_scores = base_info[column].values
        model2_scores = comp_info[column].values
        
        t_statistic, p_value = ttest_rel(model1_scores, model2_scores, alternative="two-sided")
        print(f"t-test: t-statistic = {t_statistic}, p-value = {p_value};  Column:{column}")

bimodal_mpnet_aud_sync_concat
t-test: t-statistic = -2.008846135755238, p-value = 0.11495014114274467;  Column:f1
t-test: t-statistic = 0.8095135052154954, p-value = 0.4636240204258184;  Column:Eq accuracy
t-test: t-statistic = -0.5608502723790354, p-value = 0.6048283018503621;  Column:eq opportunity
t-test: t-statistic = -2.637501010186526, p-value = 0.057731087854918685;  Column:pred equality
trimodal_mpnet_av_sync_keyw_concat
t-test: t-statistic = 0.29794948979633096, p-value = 0.7805766456143577;  Column:f1
t-test: t-statistic = -0.2118887140368696, p-value = 0.8425525812002301;  Column:Eq accuracy
t-test: t-statistic = -0.45415306374438447, p-value = 0.6732704106950728;  Column:eq opportunity
t-test: t-statistic = 0.2649581252053325, p-value = 0.8041352109971666;  Column:pred equality


In [29]:
base_model = "unimodal_mpnet_sent_keyw"
bias_models = ['unimodal_bias_oversample_mpnet_sent_keyw', 'unimodal_bias_mf-group_mpnet_sent', 'unimodal_bias_mf-mixgender_mpnet_sent', 'unimodal_bias_mf-subgroup_mpnet_sent',
                'unimodal_bias_mf-synthetic_mpnet_sent', 'unimodal_bias_mf-synthetic-mixg_mpnet_sent', 'unimodal_bias_reweighing_mpnet_sent_keyw']
columns = ["f1", "Eq accuracy", "eq opportunity", "pred equality"]

# go over each comparison model and compute the t-test
base_info = df_results[df_results["name"] == base_model]

# get all the t-test statistics
stats_info = []
for comp_name in bias_models:
    tstats = []
    comp_info = df_results[df_results["name"] == comp_name]
    
    # for each column perform the paired t-test
    for column in columns:
        model1_scores = base_info[column].values
        model2_scores = comp_info[column].values
        
        t_statistic, p_value = ttest_rel(model1_scores, model2_scores, alternative="two-sided")
        tstats.append(p_value)
        
    stats_info.append((comp_name, *tstats))

df_ttstats = pd.DataFrame(stats_info, columns=['name', *columns])
df_ttstats
        # print(f"t-test: t-statistic = {t_statistic}, p-value = {p_value};  Column:{column}")

,name,f1,Eq accuracy,eq opportunity,pred equality
0,unimodal_bias_oversample_mpnet_sent_keyw,0.642809,0.320716,0.513445,0.112155
1,unimodal_bias_mf-group_mpnet_sent,0.869067,0.216443,0.725241,0.517774
2,unimodal_bias_mf-mixgender_mpnet_sent,0.742757,0.400681,0.006968,0.242857
3,unimodal_bias_mf-subgroup_mpnet_sent,0.974011,0.708904,0.447953,0.616341
4,unimodal_bias_mf-synthetic_mpnet_sent,0.070468,0.093027,0.568631,0.057731
5,unimodal_bias_mf-synthetic-mixg_mpnet_sent,0.157157,0.452426,0.059158,0.142469
6,unimodal_bias_reweighing_mpnet_sent_keyw,0.439911,0.544897,0.727785,0.513096
